<h1><center>Jupyter notebook for Seaside (Repairs)</center></h1>
Dylan R. Sanderson<br>
Last updated: Feb. 10th, 2021

This notebook contains code to determine repair time estimates for buildings in Seaside. The building damage code must be ran before using this notebook.

Methods follow HAZUS

In [ ]:
import os
import numpy as np
import pandas as pd

### Defining repair time information
+ Building repair time information is taken from [HAZUS MH 2.1 (Earthquake Model)](https://www.fema.gov/sites/default/files/2020-09/fema_hazus_earthquake-model_technical-manual_2.1.pdf#page=613)  Section 15.2.4.
+ HAZUS provides the median values for repair times for each damage state and building type.
+ The repair time formulation follows [Kameshwar et al. (2019)](https://www.sciencedirect.com/science/article/pii/S0951832018315163?casa_token=CPiMJq2o8zAAAAAA:Hjr5X2tu2MWfcEG57JVwAMrn9QgiInDG_eoPsUAQXdZJ7VgaI3UyXVqILpD92IPTVG50R5MsaVA), in which only one set of repair time estimates are used for both earthquake and tsunami hazards and across building types.
+ Kameshwar et al. (2019) assume that a lognormal restoration model is used for buildings and that the logarithmic dispersion is 0.5.
+ Because Monte-Carlo simulation is not implemented for the optimization algorithm, the median values provided by HAZUS that are used to parameterize the lognormal restoration functions are converted to [mean values](https://www.itl.nist.gov/div898/handbook/apr/section1/apr164.htm)


| Damage State | Median | Dispersion |
| --- | --- | --- |
| Insignificant/None | 0.5 | 0.5 |
| Moderate | 60 | 0.5 |
| Heavy | 360 | 0.5 |
| Complete | 720 | 0.5 |

The repair time is thus computed as <br>
$$Repair = \sum_{ds} P_{ds} \mu_{r,ds}$$
With 
+ $P_{ds}$: probability of being in damage state $ds$
+ $\mu_{r,ds}$: mean repair time associated with damage state $ds$

In [ ]:
# median repair times for DS: None, slight, moderate, extensive, and complete
med = np.array([0.5, 60, 360, 720])
beta = np.array([0.5, 0.5, 0.5, 0.5])
DSs = ['insignific', 'moderate', 'heavy', 'complete']

# converting median/beta to mean values
mean_repair_time = med*np.exp((beta**2)/2)

### Computing earhtquake repair times

In [ ]:
path_to_earthquake_dmg = os.path.join(os.getcwd(), 'buildings_eq_1000yr.csv')
data = pd.read_csv(path_to_earthquake_dmg)
repair_save = 0
for DS_i, DS in enumerate(DSs):
    repair_save += data[DS]*mean_repair_time[DS_i]
repair_save.mean()

### Computing tsunami repair times

In [ ]:
path_to_tsunami_dmg = os.path.join(os.getcwd(), 'buildings_tsu_1000yr.csv')
data = pd.read_csv(path_to_tsunami_dmg)
repair_save = 0
for DS_i, DS in enumerate(DSs):
    repair_save += data[DS]*mean_repair_time[DS_i]
repair_save.mean()

### Computing cumulative repair times

In [ ]:
path_to_cumulative_dmg = os.path.join(os.getcwd(), 'buildings_cumulative_1000yr.csv')
data = pd.read_csv(path_to_cumulative_dmg)
repair_save = 0
for DS_i, DS in enumerate(DSs):
    repair_save += data[DS]*mean_repair_time[DS_i]
repair_save.mean()